In [7]:
import pandas as pd
from pathlib import Path

In [8]:
MAIN_FOLDER = Path().resolve()
DATA_FOLDER = MAIN_FOLDER / "data"

RAW_DATA_FOLDER = DATA_FOLDER / "raw"
TRIPS_FOLDER = RAW_DATA_FOLDER / "Concat_trips_2013-2019"
NEW_TRIPS_FOLDER = RAW_DATA_FOLDER / "10_parts_cleared"

CLEAN_DATA_FOLDER = DATA_FOLDER / "cleaned"
RIDE_VALUES_FOLDER = CLEAN_DATA_FOLDER / "ride_values"

In [9]:
import datetime


def calculate_ride_value(ride_row):
    start_time = ride_row["starttime"]
    end_time = ride_row["stoptime"]
    user_type = ride_row["usertype"].lower()

    duration_minutes = (end_time - start_time).total_seconds() / 60

    if start_time < datetime.datetime(2015, 8, 5):
        return _calculate_pre_aug_2015(duration_minutes, user_type)
    if start_time < datetime.datetime(2016, 1, 31):
        return _calculate_aug_2015_to_jan_2016(duration_minutes, user_type)
    if start_time < datetime.datetime(2017, 2, 8):
        return _calculate_jan_2016_to_feb_2017(duration_minutes, user_type)
    if start_time < datetime.datetime(2018, 2, 1):
        return _calculate_feb_2017_to_feb_2018(duration_minutes, user_type)
    if start_time < datetime.datetime(2020, 3, 24):
        return _calculate_feb_2018_to_mar_2020(duration_minutes, user_type)
    if start_time < datetime.datetime(2020, 6, 1):
        return _calculate_mar_2020_to_jun_2020(duration_minutes, user_type)
    if start_time < datetime.datetime(2020, 7, 27):
        return _calculate_jun_2020_to_jul_2020(duration_minutes, user_type)
    if start_time < datetime.datetime(2022, 1, 22):
        return _calculate_jul_2020_to_jan_2022(duration_minutes, user_type)
    if start_time < datetime.datetime(2022, 5, 10):
        return _calculate_jan_2022_to_may_2022(duration_minutes, user_type)
    if start_time < datetime.datetime(2023, 5, 10):
        return _calculate_may_2022_to_may_2023(duration_minutes, user_type)
    if start_time < datetime.datetime(2024, 2, 5):
        return _calculate_may_2023_to_feb_2024(
            duration_minutes,
            user_type,
            ride_row.get("rideable_type", "classic_bike"),
        )
    return _calculate_after_feb_2024(
        duration_minutes,
        user_type,
        ride_row.get("rideable_type", "classic_bike"),
    )


def _calculate_pre_aug_2015(duration_minutes, user_type):
    if user_type in ("member", "subscriber"):
        if duration_minutes <= 30:
            return 0.0
        if duration_minutes <= 60:
            return 1.50
        if duration_minutes <= 90:
            return 1.50 + 4.50
        additional_periods = int((duration_minutes - 90) / 30) + 1
        return 1.50 + 4.50 + (6.00 * additional_periods)

    if duration_minutes <= 30:
        return 7.0 / 2
    if duration_minutes <= 60:
        return 7.0 / 2 + 2.00
    if duration_minutes <= 90:
        return 7.0 / 2 + 2.00 + 6.00
    additional_periods = int((duration_minutes - 90) / 30) + 1
    return 7.0 / 2 + 2.00 + 6.00 + (8.00 * additional_periods)


def _calculate_aug_2015_to_jan_2016(duration_minutes, user_type):
    # 24-часовой пропуск стал стоит 9.95 долларов. Цены остались такими же.
    if user_type in ("member", "subscriber"):
        if duration_minutes <= 30:
            return 0.0
        if duration_minutes <= 60:
            return 1.50
        if duration_minutes <= 90:
            return 1.50 + 4.50
        additional_periods = int((duration_minutes - 90) / 30) + 1
        return 1.50 + 4.50 + (6.00 * additional_periods)

    if duration_minutes <= 30:
        return 9.95 / 2
    if duration_minutes <= 60:
        return 9.95 / 2 + 2.00
    if duration_minutes <= 90:
        return 9.95 / 2 + 2.00 + 6.00
    additional_periods = int((duration_minutes - 90) / 30) + 1
    return 9.95 / 2 + 2.00 + 6.00 + (8.00 * additional_periods)


def _calculate_jan_2016_to_feb_2017(duration_minutes, user_type):
    # Цены на годовой абонемент были повышены до $99. Цены остались такими же.
    _calculate_aug_2015_to_jan_2016(duration_minutes, user_type)


def _calculate_feb_2017_to_feb_2018(duration_minutes, user_type):
    # Были снижены цены для 60-90 минут. $4.5 -> $3.0 для годового и $6.0 -> $4.0 для пропуска
    if user_type in ("member", "subscriber"):
        if duration_minutes <= 30:
            return 0.0
        if duration_minutes <= 60:
            return 1.50
        if duration_minutes <= 90:
            return 1.50 + 3.00
        additional_periods = int((duration_minutes - 90) / 30) + 1
        return 1.50 + 3.00 + (6.00 * additional_periods)

    if duration_minutes <= 30:
        return 9.95 / 2
    if duration_minutes <= 60:
        return 9.95 / 2 + 2.00
    if duration_minutes <= 90:
        return 9.95 / 2 + 2.00 + 4.00
    additional_periods = int((duration_minutes - 90) / 30) + 1
    return 9.95 / 2 + 2.00 + 4.00 + (8.00 * additional_periods)


def _calculate_feb_2018_to_mar_2020(duration_minutes, user_type):
    if user_type in ("member", "subscriber"):
        if duration_minutes <= 45:
            return 0.0
        additional_periods = int((duration_minutes - 45) / 30) + 1
        return 3.00 * additional_periods

    if duration_minutes <= 180:
        return 15.0 / 2
    additional_periods = int((duration_minutes - 180) / 30) + 1
    return 15.0 / 2 + (3.00 * additional_periods)


def _calculate_mar_2020_to_jun_2020(duration_minutes, user_type):
    # Цены были снижены для одиночных поездок, которые не учитывается в датасете.
    _calculate_feb_2018_to_mar_2020(duration_minutes, user_type)


def _calculate_jun_2020_to_jul_2020(duration_minutes, user_type):
    # Я не уверен, что цены вернулись в норму, так как следующее доказательство увеличения цен появилось в июле, но пусть будет так.
    _calculate_mar_2020_to_jun_2020(duration_minutes, user_type)


def _calculate_jul_2020_to_jan_2022(duration_minutes, user_type):
    # Перешли на минутную модель по цене 0.15 долларов за минуту.
    if user_type in ("member", "subscriber"):
        if duration_minutes <= 45:
            return 0.0
        extra_minutes = duration_minutes - 45
        return 0.15 * extra_minutes

    if duration_minutes <= 180:
        return 15.0 / 2
    extra_minutes = duration_minutes - 30
    return 15.0 / 2 + 0.15 * extra_minutes


def _calculate_jan_2022_to_may_2022(duration_minutes, user_type):
    # Цены за программы повысились, но не на тарифы.
    _calculate_jul_2020_to_jan_2022(duration_minutes, user_type)


def _calculate_may_2022_to_may_2023(duration_minutes, user_type):
    # 0.16 долларов за минуту.
    if user_type in ("member", "subscriber"):
        if duration_minutes <= 45:
            return 0.0
        extra_minutes = duration_minutes - 45
        return 0.16 * extra_minutes

    if duration_minutes <= 180:
        return 15.0 / 2
    extra_minutes = duration_minutes - 30
    return 15.0 / 2 + 0.16 * extra_minutes


def _calculate_may_2023_to_feb_2024(duration_minutes, user_type, rideable_type):
    is_classic = rideable_type.lower() in ["classic_bike", "docker_bike"]
    is_electric = rideable_type.lower() == "electric_bike"
    is_scooter = rideable_type.lower() == "electric_scooter"

    if user_type in ("member", "subscriber"):
        if is_electric:
            return 0.17 * duration_minutes
        if is_scooter:
            return 0.27 * duration_minutes
        if duration_minutes <= 45:
            return 0.0
        extra_minutes = duration_minutes - 45
        return 0.17 * extra_minutes

    if is_classic:
        if duration_minutes <= 180:
            return 16.5 / 2
        extra_minutes = duration_minutes - 180
        return 16.5 / 2 + 0.17 * extra_minutes
    if is_scooter or is_scooter:
        return 16.5 / 2 + 0.42 * duration_minutes


def _calculate_after_feb_2024(duration_minutes, user_type, rideable_type):
    is_classic = rideable_type.lower() in ["classic_bike", "docker_bike"]
    is_electric = rideable_type.lower() == "electric_bike"
    is_scooter = rideable_type.lower() == "electric_scooter"

    if user_type in ("member", "subscriber"):
        if is_electric:
            return 0.18 * duration_minutes
        if is_scooter:
            return 0.29 * duration_minutes
        if duration_minutes <= 45:
            return 0.0
        extra_minutes = duration_minutes - 45
        return 0.18 * extra_minutes

    if is_classic:
        if duration_minutes <= 180:
            return 18.1 / 2
        extra_minutes = duration_minutes - 180
        return 18.1 / 2 + 0.18 * extra_minutes
    if is_scooter or is_scooter:
        return 18.1 / 2 + 0.44 * duration_minutes

In [10]:
parquet_files = [
    "trips-stations_part_1.parquet",
    "trips-stations_part_2.parquet",
    "trips-stations_part_3.parquet",
    "trips-stations_part_4.parquet",
    "trips-stations_part_5.parquet",
    "trips-stations_part_6.parquet",
    "trips-stations_part_7.parquet",
    "trips-stations_part_8.parquet",
    "trips-stations_part_9.parquet",
    "trips-stations_part_10.parquet",
]

old_trips = pd.DataFrame()
for file_name in parquet_files:
    df = pd.read_parquet(TRIPS_FOLDER / file_name)
    df["starttime"] = pd.to_datetime(df["starttime"])
    df["stoptime"] = pd.to_datetime(df["stoptime"], format="mixed")
    df["ride_value"] = df[["starttime", "stoptime", "usertype"]].apply(calculate_ride_value, axis=1)
    df.to_parquet(RIDE_VALUES_FOLDER / f"ride_value_{file_name}", index=False)
    print(f"{file_name} was processed")

    old_trips = pd.concat([old_trips, df], ignore_index=True)
df.to_parquet(RIDE_VALUES_FOLDER / f"total_2013-2019_ride_value.parquet", index=False)
print("All data was processed")

trips-stations_part_1.parquet was processed
trips-stations_part_2.parquet was processed
trips-stations_part_3.parquet was processed
trips-stations_part_4.parquet was processed
trips-stations_part_5.parquet was processed
trips-stations_part_6.parquet was processed
trips-stations_part_7.parquet was processed
trips-stations_part_8.parquet was processed
trips-stations_part_9.parquet was processed
trips-stations_part_10.parquet was processed
All data was processed


In [11]:
parquet_files = [
    "trips_part_1.parquet",
    "trips_part_2.parquet",
    "trips_part_3.parquet",
    "trips_part_4.parquet",
    "trips_part_5.parquet",
    "trips_part_6.parquet",
    "trips_part_7.parquet",
    "trips_part_8.parquet",
    "trips_part_9.parquet",
    "trips_part_10.parquet",
]

new_trips = pd.DataFrame()
for file_name in parquet_files:
    df = pd.read_parquet(NEW_TRIPS_FOLDER / file_name)
    df.rename(columns={"started_at": "starttime", "ended_at": "stoptime", "member_casual": "usertype"}, inplace=True)
    df["starttime"] = pd.to_datetime(df["starttime"])
    df["stoptime"] = pd.to_datetime(df["stoptime"], format="mixed")
    df["ride_value"] = df[["starttime", "stoptime", "usertype"]].apply(calculate_ride_value, axis=1)
    df.rename(columns={"starttime": "started_at", "stoptime": "ended_at", "usertype": "member_casual"}, inplace=True)
    df.to_parquet(RIDE_VALUES_FOLDER / f"ride_value_{file_name}_2020-2025", index=False)
    print(f"{file_name} was processed")

    new_trips = pd.concat([new_trips, df], ignore_index=True)
df.to_parquet(RIDE_VALUES_FOLDER / f"total_2020-2025_ride_value.parquet", index=False)
print("All data was processed")

trips_part_1.parquet was processed
trips_part_2.parquet was processed
trips_part_3.parquet was processed
trips_part_4.parquet was processed
trips_part_5.parquet was processed
trips_part_6.parquet was processed
trips_part_7.parquet was processed
trips_part_8.parquet was processed
trips_part_9.parquet was processed
trips_part_10.parquet was processed
All data was processed


In [12]:
new_trips["ride_value"].describe()

count    2.336765e+07
mean     3.050997e+00
std      3.892672e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      7.500000e+00
max      9.050000e+00
Name: ride_value, dtype: float64

In [13]:
import pandas as pd

# Убедимся, что started_at в формате datetime
new_trips["started_at"] = pd.to_datetime(new_trips["started_at"])

# Добавим колонку с годом
new_trips["year"] = new_trips["started_at"].dt.year

# Группировка по году и типу пользователя
grouped = (
    new_trips.groupby(["year", "member_casual"])
    .agg(
        total_ride_value=("ride_value", "sum"),
        ride_count=("ride_value", "count"),
        average_ride_value=("ride_value", "mean"),
    )
    .reset_index()
)

# Переводим суммы в миллионы и округляем
grouped["total_ride_value"] = (grouped["total_ride_value"] / 1_000_000).round(2)
grouped["average_ride_value"] = grouped["average_ride_value"].round(2)

# Общая выручка
total_by_user = new_trips.groupby("member_casual")["ride_value"].sum() / 1_000_000
total_by_user = total_by_user.round(2)

# Диапазон дат
date_range = (new_trips["started_at"].min().date(), new_trips["started_at"].max().date())

# Вывод
print(f"📅 Период данных: с {date_range[0]} по {date_range[1]}\n")

print("💰 Общая выручка по типам пользователей:")
for member_casual, value in total_by_user.items():
    print(f"  - {member_casual.capitalize()}: ${value} млн")

print("\n📊 Выручка по годам и типу пользователя:")
i = 1
for _, row in grouped.iterrows():
    print(
        f"  - {row['year']} | {row['member_casual'].capitalize():<7} → "
        f"${row['total_ride_value']} млн | "
        f"{row['ride_count']} поездок | "
        f"средняя стоимость: ${row['average_ride_value']}"
    )
    if i % 2 == 0:
        print()
    i += 1

📅 Период данных: с 2020-01-01 по 2025-02-28

💰 Общая выручка по типам пользователей:
  - Casual: $71.07 млн
  - Member: $0.23 млн

📊 Выручка по годам и типу пользователя:
  - 2020 | Casual  → $5.7 млн | 760211 поездок | средняя стоимость: $7.5
  - 2020 | Member  → $0.03 млн | 1522059 поездок | средняя стоимость: $0.02

  - 2021 | Casual  → $17.38 млн | 2317649 поездок | средняя стоимость: $7.5
  - 2021 | Member  → $0.05 млн | 2979722 поездок | средняя стоимость: $0.02

  - 2022 | Casual  → $14.14 млн | 1885226 поездок | средняя стоимость: $7.5
  - 2022 | Member  → $0.04 млн | 2617580 поездок | средняя стоимость: $0.02

  - 2023 | Casual  → $15.5 млн | 1908171 поездок | средняя стоимость: $8.12
  - 2023 | Member  → $0.05 млн | 3518214 поездок | средняя стоимость: $0.01

  - 2024 | Casual  → $17.9 млн | 1979949 поездок | средняя стоимость: $9.04
  - 2024 | Member  → $0.06 млн | 3597692 поездок | средняя стоимость: $0.02

  - 2025 | Casual  → $0.45 млн | 49599 поездок | средняя стоимость:

In [14]:
import pandas as pd

# Убедимся, что starttime в формате datetime
old_trips["starttime"] = pd.to_datetime(old_trips["starttime"])

# Добавим колонку с годом
old_trips["year"] = old_trips["starttime"].dt.year

# Группировка по году и типу пользователя
grouped = (
    old_trips.groupby(["year", "usertype"])
    .agg(
        total_ride_value=("ride_value", "sum"),
        ride_count=("ride_value", "count"),
        average_ride_value=("ride_value", "mean"),
    )
    .reset_index()
)

# Переводим суммы в миллионы и округляем
grouped["total_ride_value"] = (grouped["total_ride_value"] / 1_000_000).round(2)
grouped["average_ride_value"] = grouped["average_ride_value"].round(2)

# Общая выручка
total_by_user = old_trips.groupby("usertype")["ride_value"].sum() / 1_000_000
total_by_user = total_by_user.round(2)

# Диапазон дат
date_range = (old_trips["starttime"].min().date(), old_trips["starttime"].max().date())

# Вывод
print(f"📅 Период данных: с {date_range[0]} по {date_range[1]}\n")

print("💰 Общая выручка по типам пользователей:")
for usertype, value in total_by_user.items():
    print(f"  - {usertype.capitalize()}: ${value} млн")

print("\n📊 Выручка по годам и типу пользователя:")
i = 1
for _, row in grouped.iterrows():
    print(
        f"  - {row['year']} | {row['usertype'].capitalize():<7} → "
        f"${row['total_ride_value']} млн | "
        f"{row['ride_count']} поездок | "
        f"средняя стоимость: ${row['average_ride_value']}"
    )
    if i % 2 == 0:
        print()
    i += 1

📅 Период данных: с 2017-10-01 по 2019-12-31

💰 Общая выручка по типам пользователей:
  - Customer: $15.59 млн
  - Subscriber: $1.46 млн

📊 Выручка по годам и типу пользователя:
  - 2017 | Customer → $0.53 млн | 78087 поездок | средняя стоимость: $6.75
  - 2017 | Subscriber → $0.04 млн | 591152 поездок | средняя стоимость: $0.07

  - 2018 | Customer → $6.69 млн | 677156 поездок | средняя стоимость: $9.89
  - 2018 | Subscriber → $0.8 млн | 2925926 поездок | средняя стоимость: $0.27

  - 2019 | Customer → $8.37 млн | 880637 поездок | средняя стоимость: $9.5
  - 2019 | Subscriber → $0.62 млн | 2937367 поездок | средняя стоимость: $0.21



In [15]:
import pandas as pd

# Ensure datetime
old_trips["started_at"] = pd.to_datetime(old_trips["started_at"])

# Filter casual users
casual_rides = old_trips[old_trips["user_type"] == "casual"]

# Total number of trips
total_trips = len(casual_rides)

# Total number of unique casual users
unique_users = casual_rides["user_id"].nunique()

# Total number of days in dataset
total_days = (casual_rides["started_at"].max().normalize() - casual_rides["started_at"].min().normalize()).days + 1

# Average number of trips per user per 24 hours
avg_trips_per_user_per_day = total_trips / (unique_users * total_days)

# Output
print(f"🚲 On average, a casual member takes {avg_trips_per_user_per_day:.4f} trips per 24 hours.")

KeyError: 'started_at'